In [2]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Test database function

In [3]:
import behavior_database

In [4]:
names = [
        'Session ID',
        'Mouse ID',
        'Date',
        'Phase',
        'Left Reward Prob',
        'Right Reward Prob',
        'Block Range Min',
        'Block Range Max',
        'No. Trials',
        'No. Blocks',
        'No. Rewards',
        'p(high Port)',
        'Decision Window Duration',
        'Min Inter-trial-interval',
        'Left Solenoid Duration',
        'Right Solenoid Duration'
    ]

# Initialize data base with most recent round of experiments

Here we use the add_multi_sessions function, which assumes that the root folder contains a list of subfolders whose names are dates. Within each 'date' folder are the data files for that mouse. 

In [16]:
record_df = pd.DataFrame(data=None,columns=names,index=None)

mice = ['K1','K4','K5','K7','K9','K10','K11','K13','Q40','Q43','Q45']

In [17]:
for mouse in mice:
    directory = '/Users/shayneufeld/Dropbox/Shay_BehaviorFiles/BehaviorData_supermice/' + mouse
    record_df = behavior_database.add_multi_sessions(directory,record_df)

In [18]:
record_df.shape

(272, 16)

In [31]:
record_df[record_df['Mouse ID']=='K1']

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,01182017_K1,K1,01182017,2.0,0.9,0.1,50.0,50.0,326.0,4.0,226.0,0.74,3.0,1.0,35.0,40.0
1,01242017_K1,K1,01242017,2.0,0.9,0.1,50.0,50.0,327.0,4.0,215.0,0.71,2.0,1.0,35.0,35.0
2,01252017_K1,K1,01252017,2.0,0.1,0.9,50.0,50.0,392.0,6.0,306.0,0.83,2.0,1.0,35.0,35.0
3,01262017_K1,K1,01262017,2.0,0.9,0.1,50.0,50.0,384.0,6.0,304.0,0.86,2.0,1.0,35.0,35.0
4,01272017_K1,K1,01272017,2.0,0.9,0.1,50.0,50.0,577.0,9.0,450.0,0.88,2.0,1.0,35.0,35.0
5,01302017_K1,K1,01302017,2.0,0.9,0.1,50.0,50.0,521.0,7.0,399.0,0.85,2.0,1.0,35.0,35.0
6,01312017_K1,K1,01312017,2.0,1.0,0.0,50.0,50.0,539.0,8.0,432.0,0.88,2.0,1.0,35.0,35.0
7,02202017_K1,K1,02202017,2.0,0.9,0.1,50.0,50.0,333.0,4.0,233.0,0.77,2.0,1.0,35.0,35.0
8,02212017_K1,K1,02212017,2.0,0.9,0.1,50.0,50.0,344.0,5.0,279.0,0.87,2.0,1.0,35.0,35.0
9,02222017_K1,K1,02222017,2.0,0.1,0.9,50.0,50.0,425.0,7.0,350.0,0.89,2.0,1.0,35.0,35.0


## Add data from summer 2016 post GUI / current parameter file structure

These data were stored a bit differently. Instead of a folder for each mouse, there was a folder for each day and within in a folder for each mouse. Note to self - switching folder organization mid project is kind of stupid, although the new way is admittedly better. Anyway, no big deal. Here we can use the single 'add_session' function, and loop through the folders here in the jupyter notebook:

In [6]:
ROOT = '/Users/shayneufeld/Dropbox/Shay_BehaviorFiles/BehaviorData_Round2'

date_folders = os.listdir(ROOT)

for date in date_folders:
    
    if not date[0] == '.':
        mouse_folders = os.listdir(os.path.join(ROOT,date))

        for mouse in mouse_folders:
            if not date[0] == '.':
                curr_dir = os.path.join(ROOT,date,mouse)

                record_df = behavior_database.add_session(curr_dir,record_df)

In [7]:
record_df.shape

(316, 16)

## Add data from summer 2016 pre GUI / different parameter file structure

This is when the parameter file was a csv file structured a bit differently. I wrote a modified function called add_old_session that handles the older parameter csv file structure appropriately:

In [8]:
ROOT = '/Users/shayneufeld/Dropbox/Shay_BehaviorFiles/BehaviorData_Round2/WITHOUT_GUI'

date_folders = os.listdir(ROOT)

for date in date_folders:
    
    if not date[0] == '.':
        mouse_folders = os.listdir(os.path.join(ROOT,date))

        for mouse in mouse_folders:
            if not date[0] == '.':
                curr_dir = os.path.join(ROOT,date,mouse)

                record_df = behavior_database.add_old_session(curr_dir,record_df)

## Save record as csv file

In [9]:
record_df.to_csv(path_or_buf='/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv')

## Load in original record and add new sessions

In [23]:
old_record = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv',index_col=0)

In [25]:
new_record = old_record.append(record_df)

In [28]:
record_df.shape

(272, 16)

In [27]:
old_record.shape

(543, 16)

In [26]:
new_record.shape

(815, 16)

## Drop duplicates

In [57]:
new_record.index = np.arange(new_record.shape[0])

In [51]:
new_record[new_record['Session ID']=='11022016_K1']

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0.0,11022016_K1,K1,11022016,1.0,1.0,1.0,1000.0,1000.0,85.0,0.0,85.0,0.0,10.0,0.5,35.0,35.0
0.0,11022016_K1,K1,11022016,1.0,1.0,1.0,1000.0,1000.0,85.0,0.0,85.0,0.0,10.0,0.5,35.0,35.0


In [55]:
new_record = new_record.drop_duplicates(subset=['Session ID'])

In [59]:
new_record.to_csv(path_or_buf='/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv')